## Test the implementation of the dataset and get some basic baselines (linear classifier)

In [6]:
from datasets.SHHS_dataset_timeonly import EEGdataset
import os
os.getcwd()

'C:\\Users\\tomsm\\PycharmProjects\\thesis02\\experiments\\1D_CNN'

In [7]:
ds = EEGdataset('../../data/',1,11)

In [8]:
datapoint, label = ds.__getitem__(2)
print(datapoint.shape)

torch.Size([1, 3000])


In [14]:
labels = ds.labels
print(labels.size(0))
for lab in range(5):
    print(f"Percentage of labels that equals {lab}: {(labels==lab).sum()/labels.size(0)}")

9900
Percentage of labels that equals 0: 0.31111112236976624
Percentage of labels that equals 1: 0.02777777798473835
Percentage of labels that equals 2: 0.3847474753856659
Percentage of labels that equals 3: 0.16343434154987335
Percentage of labels that equals 4: 0.11292929202318192


In [21]:
# Train a linear classifier to set an ultimate baseline for neural network
# create the linear model classifier
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000,tol=1e-3))
# fit (train) the classifier
X1 = np.array(ds.X1.squeeze())
clf.fit(X1, ds.labels)


C:\Users\tomsm\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:705: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier())])

In [22]:
# mean accuracy of linear classifier: 56%
clf.score(X1, ds.labels)

0.5651515151515152

In [34]:
from datasets.SHHS_dataset_timeonly import EEGdataModule
import os
print(os.getcwd())
dm = EEGdataModule("../../data/", batch_size=64, data_split=[3, 1, 1], num_patients=10)
dm.setup()
print(len(dm.eeg_train.indices))
print(len(dm.eeg_val.indices))
print(len(dm.eeg_test.indices))
print(ds.__len__())

C:\Users\tomsm\PycharmProjects\thesis02\experiments\1D_CNN
5940
1980
1980
9900
